In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
from ese.experiment.analysis.inference import load_cal_inference_stats

inference_path = root / "inference/01_06_24_WMH_UncalibratedInference"

inference_info_dict = load_cal_inference_stats(
    log_dir=inference_path,
    load_image_df=True,
    load_pixel_meters_dict=False
)

In [ ]:
image_info_df = inference_info_dict['image_info_df']

In [ ]:
image_info_df.head()

In [ ]:
g = sns.catplot(
    image_info_df, 
    x="log_set", 
    y="qual_score", 
    kind="box"
)
# g.set(xlim=(0, 1), ylim=(0, 1))
g.tight_layout()
plt.show()